In [1]:
import torch
import random
from collections import deque
from torch.nn.functional import relu, prelu

In [2]:
#todo: collect all incoming edges for coalescence (search how they do it so it's consistent)
#todo: after implementing message pass algorithm, move it to forward pass (if possible... and if it doesn't overcomplicate modularly changing out attention structures...)
#todo: try graph step passing new node and computational dependency subtrees--> trim subtrees and extend undiscovered. intuition: like a net with a ball (at new node), 
# cut off at k-hop depth

# GraphSearch for single hop message passing
### new approach: graph class with nodes with child and parent links (for step and computational dependency subtree creation)
###               step() can be a random (for now uniform random) selection from children. 
#### todo: implement k-hop dependency subtree creation and think about computational efficiency... Don't want to have exponential search overhead with hops...

In [221]:
#idea: could it be possible to completely decouple the procedurality of the network and just assume it converges to something useful? 
#... Just randomly (as a function of local in/out degree) sample a layer from the network each time and just keep walking... 
#... Maybe this still allows for multi-granularity analysis, just less organized. This idea relies on some notion of convergence I think. 
#todo: inverse relations are added to the predicate set...
#todo: make training script
#todo: can enhance expressiveness by making transformation & aggregation steps 3 layer mlp's each. (in the case of the r-gcn these are the embedding layers)
# other option: can add mlp layers before and after the gnn, as pre-processing layers.
# skip connections can be used to reduce oversmoothing I.e. k-hop 3 gets preprocessed and passed past k-hop 2 as well as into it (duplication) and just gets summed together with k-hop2 outputs
# note that R-GCN uses normalized sum aggregation Also has edge dropout before batch norm
# R-GCN uses full batch adam (rmsprop+momentum) for 400 epochs
# R-GCN activation = relu, but relu-->relu-->softmax for entity classification makes sense ofcourse.
# can kind of see embeddings as eigenvectors of the implications of the structure of the graph under random walk
# check spectral node representation... its equal to the svd
# graph laplacian  adj matrix (alternative repr for adj matrix)--> decomp
# inverse relations and equality relations 

import torch
from torch.nn.functional import prelu, relu
import torch.nn as nn
from collections import defaultdict, deque
import random
EMBEDDING_SIZE = 5
DIM_W = 5
MAX_K_HOP = 3

# def message():

#end to end... some choice for type(input) in inputs module dict(input)(input)

class GCNLayer(nn.Module):
    #todo: handle size 0 batches
    def __init__(self, x_dim ,y_dim, unique_labels, num_mlp_layers=1):
        super(GCNLayer, self).__init__()
        self.edge_label_weights = nn.ModuleDict({label: nn.Linear(x_dim, y_dim) for label in unique_labels})
        self.y_dim = y_dim
        mlp_layers = []
        for _ in range(3):
            mlp_layers.append(nn.Linear(y_dim, y_dim))
            mlp_layers.append(nn.PReLU())
            mlp_layers.append(nn.Dropout(p=0.2))
            mlp_layers.append(nn.LayerNorm(y_dim))
        self.mlp = nn.Sequential(*mlp_layers)
        self.prelu = nn.PReLU()

    def forward(self, layer_node_batch, agg_method=torch.sum):
           batch_size = len(layer_node_batch)
           non_linear = []

           for i, node in enumerate(layer_node_batch):
               messages = node.collect_neighbours()
               transformed_messages = defaultdict(list)

               if messages:
                   for message in messages:
                       for parent_id, (embedding, edge_label, receiver_node_id) in message.items():
                           transformed = self.edge_label_weights[edge_label](embedding)
                           transformed_messages[i].append(transformed)

                   aggregated = agg_method(torch.stack(transformed_messages[i])) + self.mlp(node.embedding)
                   non_linear_i = self.prelu(aggregated)
               else:
                   transformed = torch.zeros(self.y_dim, dtype=torch.float32)
                   aggregated = self.mlp(node.embedding)
                   non_linear_i = self.prelu(aggregated)

               non_linear.append(non_linear_i)

           return torch.stack(non_linear)

class R_GCN(nn.Module): #change to include GCN layers and assign them batches (split them up in forward and handle them sequentially including node update)
    #it's a dependency bottleneck
    def __init__(self, x_dim, y_dim, graph, max_k_hop):
        super(R_GCN, self).__init__()
        self.graph = graph
        self.max_k_hop =max_k_hop
        self.layers = nn.ModuleList([
            GCNLayer(x_dim, y_dim, graph.unique_labels) 
            for _ in range(max_k_hop + 1)
        ])
        
    
    def forward(self, node_batch, agg_method=torch.sum): #maybe handle k_hops outside? have it be a single layer...
        mini_batch = defaultdict(list)
        
        for node, k in node_batch:
            layer = k
            mini_batch[layer].append(node)
            
        updated_embeddings = {}
        for layer in reversed(list(mini_batch.keys())):
            batch_size = len(mini_batch[layer])
            print(f'layer {layer} processing {batch_size} items...')
            batch = mini_batch[layer]
            embeddings = self.layers[layer](batch)
            for i, node in enumerate(batch):
                try:
                    updated_embeddings[node.id] = embeddings[i]
                except:
                    pass
            for node in mini_batch[layer]:
                node.embedding = updated_embeddings[node.id]
        return torch.stack([updated_embeddings[node.id] for node,k in node_batch], dim=0)


class Node:
    def __init__(self, id_):
        self.id = id_
        self.embedding = torch.rand(EMBEDDING_SIZE, dtype=torch.float32, requires_grad=True)
        self.features = {}
        self.parents = {}
        self.children = {}
        self.out_edges = []

    
    


    def collect_neighbours(self):
        local_tree = []
        for parent, labels in self.parents.items():
            for label in labels:
                local_tree.append({parent.id: (parent.embedding, label, self.id)})
        return local_tree
                
    
    def print_out_degree(self):
        out_edges = [(label,target.id) for label,target in self.out_edges]
        print(f"Out edges: {out_edges}") if out_edges else print(f'Node {self.id} has no children')

    def adjust_embeddings(self, new_embedding):
        self.embedding = new_embedding

    def add_parent(self, parent_node, label):
        if parent_node not in self.parents:
            self.parents[parent_node] = []
        self.parents[parent_node].append(label)

    def add_child(self, child_node, label):
        if child_node not in self.children:
            self.children[child_node] = []
        self.children[child_node].append(label)
        self.add_out_edge(label, child_node)

    def add_out_edge(self, label, target):
        self.out_edges.append((label, target))

    def step(self):
        if not self.children:
            return None
        edge_label, target_node = random.choice(self.out_edges)
        return (target_node, edge_label)

class Graph:
    def __init__(self):
        self.nodes = {}
        self.label_weights = {}
        self.unique_labels = set()

    def add_node(self, id_):
        if id_ not in self.nodes:
            self.nodes[id_] = Node(id_)

    def add_edge(self, from_id, to_id, label):
        if label not in self.unique_labels:
            self.unique_labels.add(label)
        if from_id not in self.nodes:
            self.add_node(from_id)
        if to_id not in self.nodes:
            self.add_node(to_id)
        
        from_node = self.nodes[from_id]
        to_node = self.nodes[to_id]
        from_node.add_child(to_node, label)
        to_node.add_parent(from_node, label)

    def initialize_label_weights(self):
        for label in list(self.unique_labels):
            self.label_weights[label] = torch.rand(DIM_W,dtype=torch.float32, requires_grad=True)
    
    def get_parents(self, id_):
        node = self.nodes.get(id_)
        return {parent.id: labels for parent, labels in node.parents.items()}

    def get_children(self, id_):
        node = self.nodes.get(id_)
        return {child.id: labels for child, labels in node.children.items()}

# pipeline: bfs_dep_tree --> loop (embedding = forward(collect_neighbours(bfs_dep_tree.get.pop()))  )
# where the first item in bfs_dep_tree list is the node and the second is the gcn layer to be used.
# collect neighbours also collects the edges for edge weights...
#todo: create inverse relations for all relations in the graph (for example by annotating them with '-' and using - weights for them. 
#set rdfs:a weights to identity matrix??

#idea save a queue snapshot when depth k-1 switches to depth k to pass with step() to next node.... i don't know how to find the edges after that...
def bfs_dep_tree(start_node, max_depth):
    queue = deque([(start_node, 0)])
    dep_tree = []
    visited = set()
    
    
    while queue:
        current_node, depth = queue.popleft()
        if depth > max_depth:
            break
        if depth == max_depth-1:
            save_queue = queue
        visited.add(current_node)
        dep_tree.append((current_node, max(1,depth)))
        for parent in current_node.parents:
            if parent not in visited:
                queue.append((parent, depth + 1))
    return dep_tree, save_queue

# def bfs_step_queue(start_queue, max_depth):

## Test some stuff:

In [222]:
graph=Graph()
nodes=[]
num_nodes = 1000
for n in range(0,num_nodes):
    graph.add_node(n)
    nodes.append(n)

preds = ["geometry:triangle","rdfs:subClassOf","FOAF:likes","rdfs:subClassOf","rdfs:domain","FOAF:knows","rdfs:isDefinedBy"]
for e in range(4000):
    obj = random.choice(nodes)
    subj = random.choice([node for node in nodes if node != obj])
    pred = random.choice(preds)
    graph.add_edge(obj,subj,pred)
graph.initialize_label_weights()

In [223]:
# A = graph.nodes[0]
# node = A
# k_hops = 3

# dep_tree = bfs_dep_tree(node, k_hops)
# # dep_tree[-1][0].parents
# # set([dep[0].id for dep in dep_tree])
# print(dep_tree)

In [ ]:
graph.initialize_label_weights()
target_node = graph.nodes[0]
k_hops = 5
r_gcn = R_GCN(x_dim=EMBEDDING_SIZE, y_dim=EMBEDDING_SIZE, graph=graph, max_k_hop=k_hops)
WALK = 50

for i in range(WALK):
    batch, step_queue = bfs_dep_tree(target_node, k_hops)
    print(f"node: {target_node.id}: embedding: {target_node.embedding}")
    updated_embedding = r_gcn(batch)
    print(f"node: {target_node.id}: embedding: {target_node.embedding}")
    try:
        target_node = target_node.step()[0]
    except:
        try:
            target_node = target_node.step()[0]
        except:
            pass

node: 0: embedding: tensor([0.4082, 0.4254, 0.3681, 0.7793, 0.3615], requires_grad=True)
layer 5 processing 1097 items...
layer 4 processing 363 items...
layer 3 processing 99 items...
layer 2 processing 24 items...
layer 1 processing 8 items...
node: 0: embedding: tensor([2.9848, 2.9848, 0.4854, 2.9848, 2.9848], grad_fn=<SelectBackward0>)
node: 593: embedding: tensor([-0.4666, -1.0078, -1.2280, -0.8221, -0.9119],
       grad_fn=<SelectBackward0>)
layer 5 processing 868 items...
layer 4 processing 275 items...
layer 3 processing 74 items...
layer 2 processing 22 items...
layer 1 processing 6 items...
node: 593: embedding: tensor([-0.3347, -0.4344, -0.2977,  0.5846,  0.5846],
       grad_fn=<SelectBackward0>)
node: 908: embedding: tensor([-3.9579, -4.6530, -4.5458, -4.5731, -4.5192],
       grad_fn=<SelectBackward0>)
layer 5 processing 1022 items...
layer 4 processing 346 items...
layer 3 processing 108 items...
layer 2 processing 31 items...
layer 1 processing 8 items...
node: 908: emb

In [210]:
node.id

48

In [105]:
# batch

In [58]:
for parent, labels in node.parents.items():
    print(parent,labels)

<__main__.Node object at 0x79ae86c5cbc0> ['rdfs:subClassOf']
<__main__.Node object at 0x79aed8f80c80> ['rdfs:isDefinedBy']
<__main__.Node object at 0x79ae86c5c1a0> ['FOAF:likes']


In [59]:
A = graph.nodes["a"]
node=A
messages = node.collect_neighbours()
if messages:
    messages = messages #placeholder for various embedding models...
    for message in messages[:1]:
        message = [m for m in message.items()]
        print(message[0][1])
        # for embedding, edge_label, receiver_node_id in message.items():
        #     print(edge_label)

(tensor([0.3756, 0.2025, 0.1175, 0.0966, 0.2818], requires_grad=True), 'rdfs:subClassOf', 'a')


In [385]:
coalesced = node.coalesce_dependencies(graph)

AttributeError: 'Node' object has no attribute 'coalesce_dependencies'

In [377]:
coalesced[0]

tensor([13.4453,  8.7500, 11.0000, 14.2422, 14.6641], dtype=torch.float16,
       grad_fn=<AddBackward0>)

In [375]:
node.print_out_degree()

Out edges: [('FOAF:knows', 'b'), ('rdfs:isDefinedBy', 'b'), ('rdfs:subClassOf', 'c'), ('rdfs:isDefinedBy', 'd'), ('rdfs:subClassOf', 'd'), ('rdfs:subClassOf', 'b'), ('FOAF:knows', 'c'), ('rdfs:isDefinedBy', 'c'), ('FOAF:likes', 'e'), ('rdfs:domain', 'e'), ('rdfs:subClassOf', 'e'), ('rdfs:subClassOf', 'e'), ('rdfs:subClassOf', 'c'), ('geometry:triangle', 'b'), ('rdfs:domain', 'e'), ('rdfs:isDefinedBy', 'c'), ('geometry:triangle', 'c'), ('rdfs:domain', 'e'), ('FOAF:likes', 'd'), ('rdfs:subClassOf', 'c')]


In [347]:
coalesced [1]

[tensor([0.0311, 0.0190, 0.0280, 0.7148, 0.1625], dtype=torch.float16,
        grad_fn=<MulBackward0>),
 tensor([0.0582, 0.2112, 0.2373, 0.2507, 0.2003], dtype=torch.float16,
        grad_fn=<MulBackward0>),
 tensor([0.0117, 0.0257, 0.4363, 0.4351, 0.1278], dtype=torch.float16,
        grad_fn=<MulBackward0>),
 tensor([0.0311, 0.0190, 0.0280, 0.7148, 0.1625], dtype=torch.float16,
        grad_fn=<MulBackward0>),
 tensor([0.0117, 0.0257, 0.4363, 0.4351, 0.1278], dtype=torch.float16,
        grad_fn=<MulBackward0>),
 tensor([0.2400, 0.0247, 0.0255, 0.5996, 0.0487], dtype=torch.float16,
        grad_fn=<MulBackward0>),
 tensor([0.1338, 0.0077, 0.0684, 0.4321, 0.0723], dtype=torch.float16,
        grad_fn=<MulBackward0>),
 tensor([0.0540, 0.0521, 0.0484, 0.4434, 0.0305], dtype=torch.float16,
        grad_fn=<MulBackward0>),
 tensor([0.0897, 0.4609, 0.0110, 0.8672, 0.4043], dtype=torch.float16,
        grad_fn=<MulBackward0>),
 tensor([0.0604, 0.3433, 0.1132, 0.5894, 0.1580], dtype=torch.fl

In [195]:
node,edge_label = node.step()
# node = graph.nodes[node_id]
node.id

'c'

In [201]:
walk = True
lambda_ = 0.01
while walk:
    node,edge_label = node.step()
    # node = graph.nodes[node_id]
    walk = random.random()>lambda_
    print(node.print_out_degree())


Out edges: [('FOAF:likes', 'b'), ('FOAF:knows', 'a'), ('FOAF:knows', 'a'), ('rdfs:subClassOf', 'a'), ('rdfs:domain', 'b'), ('FOAF:likes', 'a'), ('FOAF:knows', 'a'), ('FOAF:knows', 'a'), ('rdfs:isDefinedBy', 'b'), ('geometry:triangle', 'a'), ('rdfs:subClassOf', 'a'), ('FOAF:likes', 'a'), ('rdfs:subClassOf', 'a'), ('rdfs:subClassOf', 'a'), ('FOAF:knows', 'a'), ('rdfs:subClassOf', 'a'), ('geometry:triangle', 'a'), ('geometry:triangle', 'a'), ('rdfs:subClassOf', 'a'), ('rdfs:domain', 'b'), ('rdfs:isDefinedBy', 'a'), ('rdfs:subClassOf', 'b'), ('FOAF:knows', 'a'), ('rdfs:isDefinedBy', 'a'), ('FOAF:knows', 'a')]
None
Out edges: [('FOAF:likes', 'c'), ('FOAF:knows', 'b'), ('FOAF:knows', 'b'), ('FOAF:likes', 'b'), ('rdfs:subClassOf', 'c'), ('FOAF:knows', 'b'), ('rdfs:subClassOf', 'b'), ('rdfs:isDefinedBy', 'b'), ('rdfs:subClassOf', 'c'), ('rdfs:isDefinedBy', 'b'), ('rdfs:subClassOf', 'b'), ('rdfs:domain', 'c'), ('FOAF:knows', 'c'), ('rdfs:subClassOf', 'c'), ('rdfs:isDefinedBy', 'c'), ('rdfs:isDe

### cashew: try graphgym package :)
# Proposed GNN architecture:
## Transformation block >>>
### * linear
### * batch norm * <  my intuition is that this could replace mean() operation, just use vectorized sum() to reduce computational complexity.
### * dropout * < On linear layer in the message function
### * activation * < parametric relu = max(x,0) + alpha * min(x,0) ... alpha is trainable.
### * attention * < I'm not sure if relational weights as in RGCN fall into this category. If they are complementary in any way.
## <<< End transformation block
### * aggregation by some problem dependant function i.e. mean(), min/max/avg..._pooling(), lstm(cat(edge_embeddings)) ...
#### aggregation note: inverted degree matrix * adjacency matrix = avg(adjacency matrix)

# classical GCN:

## important design choice here... use batch norm after each layer? Normalize explicitly?
## messages = layer weight * normalized messages from prev layers
## aggregation = sum(messages) --> relu

### todo: add weighted average method. I.e. learnable row vector of size feature dim (must vectorize torch.mean explicitly for this)

In [3]:
# def aggregate(features): #this creates an aggregation directly as a matrix op
#     agg = torch.stack(features)
#     return agg, torch.mean(agg, dim=0)

# def coalesce(features):

# def normalize(messages):
#     num = len(messages)
    

# def relu(features):
#     return relu(features)

# GraphSAGE
## aggregate incoming messages (can be mean(messages, dim=0), can be a max pooling on mlp(message), can be LSTM(shuffle(messages) as mini-batch), can be sum without average (maybe this leads to batch norm later?)
## concat current node message --> relu --> send

## Uses L2 Norm as root squared error of embeddings at every layer.

# GAT

# Architectural notes:

## GCN, but vector weighting matrix is learned (which nodes to attend to and ./ unsure./ which parts of the embedding vectors/features to attend to \.\.

### how can we handle permutation invariance?

### seems that attention weights are graph conditional on search algorithm dependant

### ^ wrong. It's a function of (and on) embeddings of different node embeddings at the previous step.

### softmax ()

### parameter matrix a can be a parameter matrix on a learned single layer mlp that processes the concatenated input vectors.

### parameter matrix a is learned together with weight matrix w.

### multi-head attention, multiple relu(a) matrices.

### each a is initialized randomly, then aggregated to produce a single output

### can be parallellized worker per message.

### sparse matrix... fixed number of parameters.

# Implications/discussion

### asymmetric importance weighting

### weights are still independant of graph size, even though more complex analysis of the graph can be performed.

### graph attention mechanism scales linearly in graph size due to locality

### cool visualization of attention mechanism (implicit clustering) cora citation paper

### improved performance over GCN in some cases.




# Test stuff and transform into main() below...

In [4]:
coalesce((edge_1,edge_2, edge_3))

NameError: name 'coalesce' is not defined

In [5]:
dfs = GraphSearch(graph, protocol='BFS')

walk = True
lambda_ = 0.1
node_id = 1
stack = deque([node_id])
print(stack)
while walk:
    node_id, stack = dfs.step(stack)
    if random.random() < lambda_:
        walk = False
print(graph)

deque([1])
1


NameError: name 'coalesce' is not defined